In [1]:
from scipy import *
import numpy as np
import WLanalysis
from matplotlib import pyplot

In [2]:
from matplotlib.patches import Ellipse
def plotEllipse(pos,P,edge,ls,ilabel,ax):
    '''plot the contour for covariance p
    where position is the center position
    p is the covariance [2,2] matirx'''
    U, s, Vh = svd(P) 
    orient = math.atan2(U[1,0],U[0,0])*180/pi
    ellipsePlot = Ellipse(xy=pos, width=2.0*math.sqrt(s[0]), 
                          height=2.0*math.sqrt(s[1]), angle=orient,edgecolor=edge, fill = 0, label=ilabel,ls=ls,linewidth=1.5)
    ax.add_patch(ellipsePlot)
    return ellipsePlot

In [3]:
def Fisher(covI, der, skyscaling= 1.0 ):
    '''
    Input: 
    covariance inverse matrix (Nbin, Nbin)
    derivatives (Nparams, Nbin), where Nparams is the number of parameters 
    skyscaling = area_sims / area_actual_survey = 12.25/2e4 for our sim and LSST
    Return the Fisher matrix
    '''
    Nparams, Nbin = der.shape
    F = zeros( shape= (Nparams, Nparams))
    for i in range(Nparams):
        for j in range(Nparams):
            dA, dB = der[i], der[j]
            Mij = mat(dA).T*mat(dB) + mat(dB).T*mat(dA)  
            F [i,j]= 0.5*trace(covI*Mij)
    F /= skyscaling
    Ferr = real(sqrt(mat(F).I)) [range(Nparams), range(Nparams)]
    return F, Ferr

In [8]:
########### generate parameters for all models for emulator
########### also generate parameters for fisher derivatives

params = genfromtxt('cosmo_params_all.txt',usecols=[2,3,4])

ir = 0.05

fidu_params = (0.1, 0.3, 2.1)
params_der = array( [[[0.1*(1.0-ir/2), 0.3, 2.1],[0.1*(1.0+ir/2), 0.3, 2.1]] ,
                    [[0.1, 0.3*(1.0-ir/2), 2.1],[0.1, 0.3*(1.0+ir/2), 2.1]] ,
                    [[0.1, 0.3, 2.1*(1.0-ir/2)],[0.1, 0.3, 2.1*(1.0+ir/2)]]] )
print params_der.shape

print 'parameters used for derivatives:'
for i in range(3):
    print params_der[i][0], params_der[i][1], fidu_params[i]*ir

(3, 2, 3)
parameters used for derivatives:
[ 0.0975  0.3     2.1   ] [ 0.1025  0.3     2.1   ] 0.005
[ 0.1     0.2925  2.1   ] [ 0.1     0.3075  2.1   ] 0.015
[ 0.1     0.3     2.0475] [ 0.1     0.3     2.1525] 0.105
